In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False
    
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor    

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)

In [2]:
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

In [3]:
from torchvision.datasets import MNIST

download_root = '../data/MNIST_DATASET/'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=False)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=False)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=False)

In [4]:
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [5]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [6]:
class GRUCell(nn.Module):
    def __init__(self,input_size,hidden_size,bias=True):
        super().__init__()
        self.input_size= input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size,3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size,3*hidden_size,bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std,std)
    
    def forward(self,x,hidden):
        x = x.view(-1,x.size(1))

        gate_x = self.x2h(x)
        gate_h = self.h2h(hidden)

        # Returns a tensor with all the dimensions of input of size 1 removed.
        gate_x = gate_x.squeeze()
        gate_h = gate_h.squeeze()

        i_r,i_i,i_n = gate_x.chunk(3,1)
        h_r,h_i,h_n = gate_h.chunk(3,1)

        resetgate = F.sigmoid(i_r+h_r)
        inputgate = F.sigmoid(i_i+h_i)
        newgate = F.tanh(i_n + (resetgate*h_n))

        hy = newgate + inputgate *(hidden - newgate)
        return hy

In [7]:
class GRUModel(nn.Module):
    def __init__(self,input_dim,hidden_dim,layer_dim,output_dim,bias=True):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        self.gru_cell = GRUCell(input_dim,hidden_dim,layer_dim)
        self.fc = nn.Linear(hidden_dim,output_dim)

    
    def forward(self,x):
        h0 = Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim).cuda())
        outs = []
        hn = h0[0,:,:]

        for seq in range(x.size(1)):
            hn = self.gru_cell(x[:,seq,:],hn)
            outs.append(hn)
        out = outs[-1].squeeze()
        out = self.fc(out)

        return out


In [8]:
input_dim = 28
hidden_dim = 128
layer_dim = 1  
output_dim = 10
 
model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)
 

model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
seq_dim = 28 
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):         
        
        images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.cuda()

        loss.backward()
        optimizer.step()
        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 500 == 0:         
            correct = 0
            total = 0
            for images, labels in valid_loader:

                images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)

                correct += (predicted.cpu() == labels.cpu()).sum()

             
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

/home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration: 500. Loss: 1.6616928577423096. Accuracy: 43.59000015258789
Iteration: 1000. Loss: 0.8945668339729309. Accuracy: 76.19999694824219
Iteration: 1500. Loss: 0.29147759079933167. Accuracy: 89.7300033569336
Iteration: 2000. Loss: 0.23627926409244537. Accuracy: 93.51000213623047
Iteration: 2500. Loss: 0.032887231558561325. Accuracy: 95.05000305175781
Iteration: 3000. Loss: 0.030374981462955475. Accuracy: 95.81999969482422
Iteration: 3500. Loss: 0.1621057391166687. Accuracy: 96.33999633789062
Iteration: 4000. Loss: 0.19308774173259735. Accuracy: 96.19000244140625
Iteration: 4500. Loss: 0.05172009393572807. Accuracy: 97.0
Iteration: 5000. Loss: 0.1390017420053482. Accuracy: 97.26000213623047
Iteration: 5500. Loss: 0.08090292662382126. Accuracy: 97.62000274658203
Iteration: 6000. Loss: 0.10488361120223999. Accuracy: 97.69000244140625
Iteration: 6500. Loss: 0.0798402726650238. Accuracy: 97.80000305175781
Iteration: 7000. Loss: 0.10250391066074371. Accuracy: 97.55999755859375
Iteration:

In [10]:
def evaluate(model, val_iter):    
    corrects, total, total_loss = 0, 0, 0
    model.eval()
    for images, labels in val_iter:

        images = Variable(images.view(-1, seq_dim, input_dim).cuda())

        logit = model(images).to(device)
        loss = F.cross_entropy(logit, labels, reduction = "sum")
        _, predicted = torch.max(logit.data, 1)
        total += labels.size(0)
        total_loss += loss.item()
        corrects += (predicted == labels).sum()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [11]:
test_loss, test_acc = evaluate(model,test_loader)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument target in method wrapper_nll_loss_forward)